# Oscars Awards

In [35]:
#import
import sys
!pip install pyspark

    100% |████████████████████████████████| 213.4MB 95kB/s  eta 0:00:01 1% |▍                               | 2.6MB 4.0MB/s eta 0:00:54    3% |█▏                              | 8.0MB 8.7MB/s eta 0:00:24    6% |██                              | 14.0MB 3.6MB/s eta 0:00:56    10% |███▍                            | 22.4MB 6.8MB/s eta 0:00:29    10% |███▌                            | 23.4MB 10.6MB/s eta 0:00:18    17% |█████▊                          | 37.8MB 11.5MB/s eta 0:00:16    18% |█████▉                          | 39.2MB 9.4MB/s eta 0:00:19    46% |██████████████▉                 | 98.8MB 11.6MB/s eta 0:00:10    52% |████████████████▊               | 111.4MB 4.0MB/s eta 0:00:26    62% |████████████████████            | 133.9MB 7.9MB/s eta 0:00:11    69% |██████████████████████▏         | 147.8MB 25.4MB/s eta 0:00:03    70% |██████████████████████▋         | 150.7MB 13.2MB/s eta 0:00:05    72% |███████████████████████         | 153.8MB 12.1MB/s eta 0:00:05    78% |████████████████████

In [1]:
import scraperwiki
import requests
from lxml import html
import pandas
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import numpy
from pyspark.sql.functions import split, explode

## Web Scraping

In [2]:
url= "http://awardsdatabase.oscars.org/Search/GetResults?query=%7B%22FieldFilters%22:[%7B%22Index%22:1,%22Field%22:%7B%22Id%22:3,%22DisplayName%22:%22Award%20Category%22,%22FieldName%22:%22name%22,%22SortViewFieldName%22:%22Category%22,%22SortViewSortCode%22:%220%22,%22IsSortable%22:true,%22SortBy%22:%22Alpha%22%7D,%22FieldValues%22:[%221%22,%222%22,%223%22,%224%22,%2210%22,%2225%22]%7D],%22IsWinnersOnly%22:1,%22AwardShowFrom%22:1,%22AwardShowTo%22:90,%22Sort%22:%223-Award%20Category-Chron%22,%22Search%22:%22Advanced%22%7D"

#params= {"FieldFilters":[{'Index':1,'Field':{'Id':3,'DisplayName':'Award%20Category','FieldName':'name','SortViewFieldName':'Category','SortViewSortCode':'0','IsSortable':'true','SortBy':'Alpha'},'FieldValues':['1','2','3','4','10','25']}],'IsWinnersOnly':1,'AwardShowFrom':1,'AwardShowTo':90,'Sort':'3-Award Category-Alpha','Search':'Advanced'}
#website = requests.get(url, params=params)

browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get(url)
root = html.fromstring(browser.page_source)


Checking for mac64 chromedriver:2.45 in cache
Driver found in /Users/carmenaguilargarcia/.wdm/chromedriver/2.45/mac64/chromedriver


In [3]:
rows= root.cssselect('div.awards-result-chron.result-group.group-awardcategory-chron')
print(len(rows))

90


In [4]:
results = []
for r in rows:
    Year = r.cssselect("div.result-group-title a.nominations-link")[0].text_content()
    #print(Year)
    for i in range(0,8):
        try:
            Category = r.cssselect('div.result-subgroup-title a.nominations-link')[i].text_content()
            #print(Category)
            Person = r.cssselect('div.awards-result-nominationstatement a.nominations-link')[i].text_content()
            Link_person = r.cssselect('div.awards-result-nominationstatement a.nominations-link')[i].get('href')
            #print(Person)
            #print(Link_person)
            
            if r.cssselect('div.awards-result-nominations.awards-result-nominations-actingorsimilar'):
                Film=r.cssselect('div.awards-result-nominations.awards-result-nominations-actingorsimilar')[i].text_content()
            else:
                Film = r.cssselect('div.awards-result-film a.nominations-link')[i].text_content()
            #print(Film)
            
        except: IndexError
            #continue
        results.append((Year,Category,Person,Link_person,Film))


In [5]:
len(results)

720

In [6]:
df = pandas.DataFrame(results,columns=["Year","Category","Person", "Link_person","Film"])

In [7]:
df.head()

,Year,Category,Person,Link_person,Film
0,1927/28 (1st),ACTOR,Emil Jannings,/Search/Nominations?nominationId=3&view=1-Nomi...,\n ...
1,1927/28 (1st),ACTRESS,Janet Gaynor,/Search/Nominations?nominationId=6&view=1-Nomi...,\n ...
2,1927/28 (1st),DIRECTING (Comedy Picture),Lewis Milestone,/Search/Nominations?nominationId=19&view=1-Nom...,\n ...
3,1927/28 (1st),DIRECTING (Dramatic Picture),Frank Borzage,/Search/Nominations?nominationId=21&view=1-Nom...,\n ...
4,1927/28 (1st),WRITING (Adaptation),Benjamin Glazer,/Search/Nominations?nominationId=34&view=1-Nom...,\n ...


In [240]:
oscars = df.drop_duplicates() #having a quick look at my dataset, I saw that I had some rows duplicated
len(oscars)

631

In [178]:
oscars.to_csv("oscars1.csv")

In [241]:
oscars.loc[:,"Person"] = oscars['Person'].str.replace("(Written by |Screenplay by |Screenplay and |Story by |Adaptation by|Dialogue by | stories by |Written for the screen by )", "")

#no idea how to avoid the warning message but it is working

In [242]:
pandas.options.display.max_rows = 650
len(oscars)
oscars.head(n=10)

,Year,Category,Person,Link_person,Film
0,1927/28 (1st),ACTOR,Emil Jannings,/Search/Nominations?nominationId=3&view=1-Nomi...,\n ...
1,1927/28 (1st),ACTRESS,Janet Gaynor,/Search/Nominations?nominationId=6&view=1-Nomi...,\n ...
2,1927/28 (1st),DIRECTING (Comedy Picture),Lewis Milestone,/Search/Nominations?nominationId=19&view=1-Nom...,\n ...
3,1927/28 (1st),DIRECTING (Dramatic Picture),Frank Borzage,/Search/Nominations?nominationId=21&view=1-Nom...,\n ...
4,1927/28 (1st),WRITING (Adaptation),Benjamin Glazer,/Search/Nominations?nominationId=34&view=1-Nom...,\n ...
5,1927/28 (1st),WRITING (Original Story),Ben Hecht,/Search/Nominations?nominationId=36&view=1-Nom...,\n ...
6,1927/28 (1st),WRITING (Title Writing),Joseph Farnham,/Search/Nominations?nominationId=38&view=1-Nom...,\n ...
7,1927/28 (1st),SPECIAL AWARD,Joseph Farnham,/Search/Nominations?nominationId=38&view=1-Nom...,\n ...
8,1928/29 (2nd),ACTOR,Warner Baxter,/Search/Nominations?nominationId=43&view=1-Nom...,In Old Arizona
9,1928/29 (2nd),ACTRESS,Mary Pickford,/Search/Nominations?nominationId=44&view=1-Nom...,Coquette


In [105]:
#testing block
''''
newdataframe = oscars
newdataframe = newdataframe.reset_index()

In [106]:
len(newdataframe)

631

In [109]:
#testing block
''''
splitted = newdataframe.Person.str.split("(, |; |&| and )")
lens= splitted.str.len()

print(len(splitted))
print(len(lens))

df2 = newdataframe.loc[newdataframe.index.repeat(lens)].assign(Person_split=numpy.concatenate(splitted))
delete= [",", ";", "&"]
df2 = df2[~df2.Person_split.str.contains('|'.join(delete))]
print(len(df2))

631
631
730


In [243]:
oscars = oscars.reset_index()
splitted_person = oscars.Person.str.split("(, |; |&| and )")
lens2= splitted_person.str.len()
oscars = oscars.loc[oscars.index.repeat(lens2)].assign(Person_split=numpy.concatenate(splitted_person))

In [244]:
delete= [",", ";", "&"]
oscars = oscars[~oscars.Person_split.str.contains('|'.join(delete))]
print(len(oscars))

730


In [245]:
#https://www.thedataschool.co.uk/rachel-costa/scraping-wikipedia/
#https://www.wikidata.org/wiki/Wikidata:List_of_properties/human
listQnumber=[]
for i in (oscars.Person_split.str.replace(" ","_")):
    wikiurl = "https://www.wikidata.org/w/api.php?"
    params = {
    "action":"wbsearchentities",
    "search": i,
    "format":"json",
    "language": "en"
}
    rewiki1 = requests.get(wikiurl, params=params)
    try:
        Qnumber = rewiki1.json()['search'][0]['title']
    except: IndexError
    listQnumber.append(Qnumber)


In [246]:
len(listQnumber)

730

In [247]:
oscars["Qnumber"]=listQnumber

In [248]:
personaldata1 = []
personaldata2 = []
personaldata3 = []
for q in listQnumber:
        params2 = {
    "action":"wbgetentities",
    "ids": q,
    "format":"json",
    "languages": "en"
}
        rewiki2 = requests.get(wikiurl, params=params2)
        try:
            Birthdate=rewiki2.json()['entities'][q]['claims']['P569'][0]['mainsnak']['datavalue']['value']['time']
            Gender=rewiki2.json()['entities'][q]['claims']['P21'][0]['mainsnak']['datavalue']['value']['id']
            Placebirth=rewiki2.json()['entities'][q]['claims']['P19'][0]['mainsnak']['datavalue']['value']['id']
        except KeyError:
            Birthdate="Error"
            Gender="Error"
            Placebirth="Error"
        personaldata1.append(Birthdate)
        personaldata2.append(Gender)
        personaldata3.append(Placebirth)
    

In [249]:
oscars["date_of_birth"]=personaldata1
oscars["gender_code"]=personaldata2
oscars["Placebirth_code"]=personaldata3

In [250]:
len(oscars)

730

In [251]:
gender2 = []
for g in oscars["gender_code"]:
    wiki_url_id = "https://www.wikidata.org/w/api.php?"
    param_id={
        "action":"wbgetentities",
        "ids":g,
        "format":"json"
    }
    rewiki_url = requests.get(wiki_url_id, params=param_id)
    try:
        gender_data = rewiki_url.json()["entities"][g]["labels"]["en"]["value"]
    except KeyError:
        gender_data="Error"
    gender2.append(gender_data)

In [252]:
oscars["gender"]=gender2

In [253]:
len(oscars)

730

In [254]:
Placebirth = []
for c in oscars["citizen_code"]:
    wiki_url_id = "https://www.wikidata.org/w/api.php?"
    param_id={
        "action":"wbgetentities",
        "ids":c,
        "format":"json"
    }
    rewiki_url_cit = requests.get(wiki_url_id, params=param_id)
    try:
        Placebirth_data = rewiki_url_cit.json()["entities"][c]["aliases"]["en"][0]["value"]
    except KeyError:
        citizen_data="Error"
    Placebirth.append(Placebirth_data)

In [255]:
len(citizen)

730

In [256]:
oscars["Placebirth"]=Placebirth

In [259]:
oscars_dem = oscars

In [260]:
oscars_dem[["Year","Edition"]]=oscars_dem['Year'].str.split(" ", expand=True)

In [261]:
#1932/33 last year of both years
oscars['Year']=oscars['Year'].str.replace(r"[0-9][0-9]/", "")

In [262]:
len(oscars_dem)

730

In [263]:
oscars_dem['date_work']=oscars_dem['date_of_birth']

In [264]:
oscars_dem.loc[:,"date_work"]=oscars_dem['date_work'].str.replace("+","")
oscars_dem.loc[:,"date_work"]=oscars_dem['date_work'].str.replace("T00:00:00Z","")

In [265]:
len(oscars_dem)

730

In [277]:
oscars_dem[["yearbirth","monthbirth","daybirth"]] = oscars_dem['date_work'].str.split("-", expand=True)

In [279]:
len(oscars_dem)

730

In [282]:
O=[int(Y) for Y in oscars_dem['Year']]
N=[]
for d in oscars_dem['yearbirth']:
    try:
        N_d=int(d)
    except: ValueError
    N.append(N_d)

In [287]:
oscars_dem['Year']=O
oscars_dem['yearbirth']=N

In [284]:
len(oscars_dem)

730

In [288]:
oscars_dem['age_award']=oscars_dem['Year']-oscars_dem['yearbirth']

In [290]:
oscars_dem.head()

,index,Year,Category,Person,Link_person,Film,Person_split,Qnumber,date_of_birth,gender_code,citizen_code,gender,citizen,Edition,date_work,yearbirth,monthbirth,daybirth,date_of_birth0,age_award
0,0,1928,ACTOR,Emil Jannings,/Search/Nominations?nominationId=3&view=1-Nomi...,\n ...,Emil Jannings,Q77181,+1884-07-23T00:00:00Z,Q6581097,Q68250,male,"Rorschach, St. Gallen",(1st),1884-07-23,1884,07,23,1884,44
1,1,1928,ACTRESS,Janet Gaynor,/Search/Nominations?nominationId=6&view=1-Nomi...,\n ...,Janet Gaynor,Q182462,+1906-10-06T00:00:00Z,Q6581072,Q1345,female,Philly,(1st),1906-10-06,1906,10,06,1906,22
2,2,1928,DIRECTING (Comedy Picture),Lewis Milestone,/Search/Nominations?nominationId=19&view=1-Nom...,\n ...,Lewis Milestone,Q285928,+1895-09-30T00:00:00Z,Q6581097,Q21197,male,Chisinau,(1st),1895-09-30,1895,09,30,1895,33
3,3,1928,DIRECTING (Dramatic Picture),Frank Borzage,/Search/Nominations?nominationId=21&view=1-Nom...,\n ...,Frank Borzage,Q369190,+1894-04-23T00:00:00Z,Q6581097,Q23337,male,Salt Lake,(1st),1894-04-23,1894,04,23,1894,34
4,4,1928,WRITING (Adaptation),Benjamin Glazer,/Search/Nominations?nominationId=34&view=1-Nom...,\n ...,Benjamin Glazer,Q817511,+1887-05-07T00:00:00Z,Q6581097,Q10686,male,Béal Feirste,(1st),1887-05-07,1887,05,07,1887,41


In [291]:
oscars.to_excel("oscars_dem.xlsx")

## New categories

In [3]:
newoscars = pandas.read_excel('oscars_copia.xlsx')

In [14]:
newoscars.head()

,index,Year,Category,Person,Link_person,Film,Person_split,Qnumber,date_of_birth,gender_code,citizen_code,gender,citizen,Edition,date_work,yearbirth,monthbirth,daybirth,age_award
0.0,0.0,1928,ACTOR,Emil Jannings,/Search/Nominations?nominationId=3&view=1-Nomi...,"The Last Command {""General Dolgorucki [Grand D...",Emil Jannings,Q77181,+1884-07-23T00:00:00Z,Q6581097,Q68250,male,"Rorschach, St. Gallen",(1st),1884-07-23,1884.0,7.0,23.0,44.0
1.0,1.0,1928,ACTRESS,Janet Gaynor,/Search/Nominations?nominationId=6&view=1-Nomi...,"7th Heaven {""Diane""}; StreetAngel {""Angela""};a...",Janet Gaynor,Q182462,+1906-10-06T00:00:00Z,Q6581072,Q1345,female,Philly,(1st),1906-10-06,1906.0,10.0,6.0,22.0
2.0,2.0,1928,DIRECTING (Comedy Picture),Lewis Milestone,/Search/Nominations?nominationId=19&view=1-Nom...,Two Arabian Knights,Lewis Milestone,Q285928,+1895-09-30T00:00:00Z,Q6581097,Q21197,male,Chisinau,(1st),1895-09-30,1895.0,9.0,30.0,33.0
3.0,3.0,1928,DIRECTING (Dramatic Picture),Frank Borzage,/Search/Nominations?nominationId=21&view=1-Nom...,7th Heaven,Frank Borzage,Q369190,+1894-04-23T00:00:00Z,Q6581097,Q23337,male,Salt Lake,(1st),1894-04-23,1894.0,4.0,23.0,34.0
4.0,4.0,1928,WRITING (Adaptation),Benjamin Glazer,/Search/Nominations?nominationId=34&view=1-Nom...,7th Heaven,Benjamin Glazer,Q817511,+1887-05-07T00:00:00Z,Q6581097,Q10686,male,Béal Feirste,(1st),1887-05-07,1887.0,5.0,7.0,41.0


In [30]:
citizenship_code=[]
for i in newoscars["Qnumber"]:
    wiki_url_id = "https://www.wikidata.org/w/api.php?"
    param_id={
        "action":"wbgetentities",
        "ids":i,
        "format":"json"
    }
    rewiki3 = requests.get(wiki_url_id, params=param_id)
    try:
        #ethnic_data = rewiki3.json()['entities'][e]['claims']['P172'][0]['mainsnak']['datavalue']['value']['id']
        citizenship_data= rewiki3.json()['entities'][i]['claims']['P27'][0]['mainsnak']['datavalue']['value']['id']
    except KeyError:
        #ethnic_data="Error"
        citizenship_data="Error"
    citizenship_code.append(citizenship_data)


In [32]:
citizen = []
for i in citizenship_code:
    wiki_url_id = "https://www.wikidata.org/w/api.php?"
    param_id={
        "action":"wbgetentities",
        "ids":i,
        "format":"json"
    }
    rewiki_country = requests.get(wiki_url_id, params=param_id)
    try:
        data_cit = rewiki_country.json()["entities"][i]["aliases"]["en"][0]["value"]
    except KeyError:
        data_cit="Error"
    citizen.append(data_cit)

In [33]:
citizen

['FRG',
 'U.S.A.',
 'Tsarist Russia',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'Dominion of Canada',
 '🇬🇧',
 'FRG',
 '🇬🇧',
 'Dominion of Canada',
 'Tsarist Russia',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 '🇬🇧',
 'U.S.A.',
 '🇬🇧',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'fr',
 'Regno d’Italia',
 'U.S.A.',
 'U.S.A.',
 '🇬🇧',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'FRG',
 'U.S.A.',
 'Regno d’Italia',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'FRG',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'FRG',
 'hu',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'Regno d’Italia',
 'U.S.A.',
 'U.S.A.',
 'Éire',
 'Éire',
 'U.S.A.',
 'Republic of South Africa',
 '🇬🇧',
 'U.S.A.',
 '🇬🇧',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'U.S.A.',
 'hu',
 'U.S.A.'

In [34]:
newoscars["nationality"]=citizen

In [35]:
newoscars.to_excel("newoscars.xlsx")

In [23]:
!pip install wikipedia

  Running setup.py bdist_wheel for wikipedia ... done
  Stored in directory: /Users/carmenaguilargarcia/Library/Caches/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [24]:
import wikipedia

In [27]:
wikipedia.search("George Henry Sanders")

['George Sanders',
 'Henry Russell Sanders',
 'Henry Sanders (historian)',
 'Jay O. Sanders',
 'Bernie Sanders',
 'Tom Conway',
 'Sanders (surname)',
 'The Saint (film series)',
 'The Larry Sanders Show',
 'Corey Sanders']